In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML 

In [24]:
df = pd.read_csv("raw_results/xgb_p_h_0.0/results_0.csv")
# df = df[["A", "B"]]
pd.crosstab(df.A, df.B)

B,0,1
A,,
0,3639,599
1,947,4657


In [25]:
df = pd.read_csv("raw_results/xgb_p_h_1.0/results_0.csv")
# df = df[["A", "B"]]
pd.crosstab(df.A, df.B)

B,0,1
A,,
0,3524,1249
1,699,4370
